<a href="https://colab.research.google.com/github/mHassan11/Reddit_PushshiftAPI/blob/master/Pushshift_Reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import requests
import json
import csv
import time
import datetime

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
file_directory = "/content/gdrive/My Drive/Reddit Data/dataset/"

In [0]:
def getPushshiftData(after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?'+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [0]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    full_link = subm['full_link']
    author = subm['author']

    selftext = subm['selftext']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id,title,url,full_link,author,selftext,
                    score,created,numComms,permalink,flair))
    subStats[sub_id] = subData

In [0]:
#Subreddit to query
sub='news'
#before and after dates

before = "1576022400" #Dec 1st 2019 0h0m0s
after = "1575158400"  #Dec 10th 2019 24h0m0s

subCount = 0
subStats = {}

data = getPushshiftData( after, before, sub)# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData( after, before, sub)
    
print(len(data))

In [0]:
subCount

In [0]:
def updateSubs_file():
    upload_count = 0
    location = file_directory 
    print("location: ", location)
    sub_red = sub
    directory_name = location + sub_red +'/'
    
    file = directory_name+sub_red+ '.csv'
    print("file: ",file)
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Full Link","Author","Self Text","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
updateSubs_file()